# Import packages

In [1]:
import logging
import os
import sys
import tempfile
from glob import glob

import torch
from PIL import Image
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

import monai
from monai.data import create_test_image_2d, list_data_collate, decollate_batch
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from monai.networks.nets import UNet
from monai.transforms import (
    Activations,
    AddChanneld,
    AsDiscrete,
    Compose,
    LoadImaged,
    RandCropByPosNegLabeld,
    RandAdjustContrastd,
    RandRotate90d,
    ScaleIntensityd,
    EnsureTyped,
    EnsureType,
    AsChannelFirstd,
    AsChannelLast,
    Resized,
    RandScaleCropd,
    RandRotated,
    Rotated,
    SaveImage,
    ThresholdIntensity,
    ThresholdIntensityd,
    RandBiasField
)

# Check MONAI configurations

In [2]:
monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

MONAI version: 0.8.1
Numpy version: 1.20.3
Pytorch version: 1.11.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 71ff399a3ea07aef667b23653620a290364095b1

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: NOT INSTALLED or UNKNOWN VERSION.
scikit-image version: 0.18.3
Pillow version: 8.4.0
Tensorboard version: 2.8.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.12.0
tqdm version: 4.62.3
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.8.0
pandas version: 1.3.4
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



# Process VGH data

In [3]:
# Set the Data folder
data_path = "D:/nycu_deep_learning/SEG_Train_Datasets/"

## -obtain testing data list

In [4]:
# Load testing files

tempdir = data_path + "Public_Image/"
test_images = sorted(glob(os.path.join(tempdir, "*.jpg")))

tempdir = data_path + "mask_img/"
test_segs = sorted(glob(os.path.join(tempdir, "*.png")))

test_files = [{"img": img, "seg": seg} for img, seg in zip(test_images[:], test_segs[:131])]


# Define Transform for image and Segmentation

In [5]:
# define transforms for image and segmentation
threshold_value = 0.4
cval_value=0.9
test_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        
        AddChanneld(keys=["seg"]),        
        AsChannelFirstd(keys=["img"]),

        ScaleIntensityd(keys=["img", "seg"]),
        #Resized(keys=["img", "seg"], spatial_size=[800, 800]),
        EnsureTyped(keys=["img", "seg"]),
        #ThresholdIntensityd(keys=["img"],threshold=threshold_value,above=False,cval=cval_value)
        
    ]
)
test_ds = monai.data.Dataset(data=test_files, transform=test_transforms)
post_trans = Compose([EnsureType(), Activations(sigmoid=True), AsDiscrete(threshold=0.5)])

# Create Data Loader, Save Output, Model Architecture

In [6]:
# sliding window inference need to input 1 image in every iteration
test_loader = DataLoader(test_ds, batch_size=1, num_workers=4, collate_fn=list_data_collate)
dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
post_trans = Compose([EnsureType(), Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
saver = SaveImage(output_dir="./output", output_ext=".png",scale=255,separate_folder=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = monai.networks.nets.DynUNet(
    spatial_dims=2,
    in_channels=3,
    out_channels=1,
    kernel_size=(3,3,3,3,3,3),
    strides=(1,2,2,2,2,2),
    filters  = (36, 64, 128, 256, 512, 1024), #  [32, 64, 128, 256, 512, 1024][: len(strides)]
    upsample_kernel_size=(2,2,2,2,2,2), # The values should equal to strides[1:]
    res_block=True,
    trans_bias=True,
).to(device)

# Load previous model

In [7]:
model = torch.nn.DataParallel(model)

In [8]:
model.load_state_dict(torch.load("best_metric_model_segmentation2d_dict_15.pth"))

<All keys matched successfully>

In [9]:
#model.load_state_dict(torch.load("Final_model_40_epoches_segmentation2d_dict.pth"))

In [10]:
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 16))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image,'gray')
    plt.show()

# Run evaluation on testing data

In [12]:
import cv2
import numpy as np
path = r'C:\Users\chwu\nycu_deep_learning\output'
os.mkdir(path)

In [13]:
os.chdir(path)

In [1]:
model.eval()

with torch.no_grad():
    for test_data in test_loader:
        test_images, test_labels = test_data["img"].to(device), test_data["seg"].to(device)

        # define sliding window size and batch size for windows inference
        roi_size = (1696, 928)
        sw_batch_size = 2
        test_outputs = sliding_window_inference(test_images, roi_size, sw_batch_size, model)

        visualize( 
            image=test_images[0].cpu().permute(1,2,0), 
            ground_truth_mask=test_labels[0].cpu().permute(1,2,0), 
            predicted_mask=test_outputs[0].squeeze().cpu().numpy().round()
        )   
       
        test_outputs = [post_trans(i) for i in decollate_batch(test_outputs)]
        test_labels = [post_trans(i) for i in decollate_batch(test_labels)]
        
        #test_labels = decollate_batch(test_labels)
        # compute metric for current iteration
        dice_metric(y_pred=test_outputs, y=test_labels)
        for test_output in test_outputs:   
            image_names = test_data['img_meta_dict']['filename_or_obj'][0].split('\\')[4][:-4]
            test_output = test_output.permute(0,2,1)
            test_output_copy = test_output.cpu().numpy()
            cv2.imwrite(image_names+'.png' , (test_output_copy*255).squeeze())
#             saver = SaveImage(output_dir="./output", output_postfix = '' ,output_ext=".png",scale=255,separate_folder=False)
#             saver(test_output*255,meta_data=' ')
    # aggregate the final mean dice result    
    print("evaluation metric:", dice_metric.aggregate().item())
    # reset the status
    dice_metric.reset()

NameError: name 'model' is not defined